In [88]:
#--------------------------------------------------
#  LOAD PACKAGES
#--------------------------------------------------

require(readxl)
library(tidyr)
library(tidyverse)

#--------------------------------------------------
#  LOAD FUNCTIONS
#--------------------------------------------------

setwd('C:/Users/fb428/OneDrive - University of Exeter/Documents/Tropoz/')
source('scripts/POD_functions.R')

f = 'BR1'

parts_list <- c('Leaves_dw', 'Petiole_dw', 'Stems_dw', 'Roots_dw', 'Lamina_dw')

#--------------------------------------------------
#  LOAD DATA
#--------------------------------------------------

csvs <- fileload('DOSEinputs', '_1p')
biomass <- read_excel(gsub(" ", "", paste(f, '/', f, "_Data-good.xlsx")), sheet="Data")

POD <- POD_extract(csvs)
                      
#--------------------------------------------------
#  MEAN BIOMASS AND ADD TO DATAFRAME
#--------------------------------------------------                        

SD_biomass <- aggregate( Biomass ~ Cham_ID, biomass, sd )
names(SD_biomass)[2] <- 'Biomass.sd'
Avg_biomass <- aggregate( Biomass ~ Cham_ID, biomass, mean )

cat('\n\n Biomass column names: \n')
print(colnames(biomass))

# Get column index of all the plant parts

plant_part_index <- c(0)
for (i in 1:5) {
    
    index <- which(colnames(biomass) == parts_list[i])
    plant_part_index<- c(plant_part_index, index)
    }

# Mean plant parts

Plant_parts_mean <- aggregate( biomass[plant_part_index], by = list(biomass$Cham_ID), FUN = mean)
names(Plant_parts_mean)[1] <- 'Cham_ID'
Plant_parts_sd <- aggregate( biomass[plant_part_index], by = list(biomass$Cham_ID), FUN = sd)
sd_names <- c('Cham_ID') 
for (i in 2:length(colnames(Plant_parts_sd))) {
    
    name <- gsub(" ", "", paste(colnames(Plant_parts_sd)[i], '_sd'))
    sd_names <- c(sd_names, name)
    }
names(Plant_parts_sd) <- sd_names

# Merge all

Dose_list <- list(POD, Avg_biomass, SD_biomass, Plant_parts_mean, Plant_parts_sd)
Dose <- Dose_list %>% reduce(full_join, by='Cham_ID')

cat('\n\n new dataframe column names: \n')
print(colnames(Dose))

#----------------------SAVE DATAFRAME------------------------

write.csv(Dose, gsub(" ", "", paste(f,'/script_output_', f, '/', f, '_POD-biomass_p.csv')), row.names=FALSE)

#------------------------------------------------------------
# GET INTERCEPT AND GRADIENT FOR YIELD CALCULATIONS
#------------------------------------------------------------

PODy <- yield_features(Dose, Dose$PODy, 'PODy')
POD0 <- yield_features(Dose, Dose$POD0, 'POD0')
AOT40 <- yield_features(Dose, Dose$AOT40, 'AOT40')

df_list <- list(PODy, POD0, AOT40)

yield_df <- Reduce(function(x, y) merge(x, y, all=TRUE), df_list)
                   
#yield_df$POD_type <- c('PODy', 'POD0', 'AOT40')
yield_df$Exp <- c(f, f, f)
                   
cat('\n\n summary dataframe:')
yield_df
              
#----------------------SAVE DATAFRAME------------------------
                   
write.csv(yield_df, gsub(" ", "", paste(f,'/script_output_', f, '/', f, '_yield_slope_p.csv')), row.names=FALSE)



 Biomass column names: 
 [1] "Exp_ID"         "Plant_ID"       "Cham_ID"        "Initial_Height"
 [5] "Initial_Leaf"   "Initial_D1"     "Initial_D2"     "Final_ height" 
 [9] "Final_D1"       "Final_D2"       "Final_Leaf"     "Leaf_fw6"      
[13] "Leaf_fw3"       "Leaf_dw3"       "Date_target"    "Lamina_dw"     
[17] "Leaves_dw_bag"  "Petiole_dw_bag" "Stems_dw_bag"   "Roots_dw_bag"  
[21] "Leaves_dw"      "Petiole_dw"     "Stems_dw"       "Roots_dw"      
[25] "AGB"            "Biomass"        "LA_3"           "LA_lamina"     
[29] "LA_Av"          "LMA"            "LMA_lamina"     "Notes"         


Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in mean.default(X[[i]], ...):
"argument is not numeric or logical: returning NA"Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
"NAs introduced by coercion"Warning messag



 new dataframe column names: 
 [1] "Cham_ID"       "PODy"          "POD0"          "AOT40"        
 [5] "Biomass"       "Biomass.sd"    "Leaves_dw"     "Petiole_dw"   
 [9] "Stems_dw"      "Roots_dw"      "Lamina_dw"     "Leaves_dw_sd" 
[13] "Petiole_dw_sd" "Stems_dw_sd"   "Roots_dw_sd"   "Lamina_dw_sd" 


 summary dataframe:

r.squared,Intercept,Slope,Yield_relationship,POD_type,Exp
0.5232206,75.89763,-0.3218320,-0.004240344,AOT40,BR1
0.5830584,79.85269,-0.8126380,-0.010176714,PODy,BR1
0.5836113,81.96232,-0.8135817,-0.009926289,POD0,BR1


In [89]:
Dose

Cham_ID,PODy,POD0,AOT40,Biomass,Biomass.sd,Leaves_dw,Petiole_dw,Stems_dw,Roots_dw,Lamina_dw,Leaves_dw_sd,Petiole_dw_sd,Stems_dw_sd,Roots_dw_sd,Lamina_dw_sd
1,4.707582,7.314495,0.08530351,72.15874,4.478803,34.71224,7.5680,17.1755,12.7030,NA,4.278261,1.1669047,2.280371,2.537197,NA
2,8.366810,10.949322,6.42648220,73.57687,4.788998,31.67287,7.5880,20.3030,14.0130,NA,3.252935,1.8610929,1.904985,2.010531,NA
3,15.637544,18.225096,26.00280952,71.54031,5.545662,31.24381,6.9405,18.7930,14.5630,NA,2.903076,0.9979103,1.382425,2.987424,NA
4,2.302390,4.866062,0.00000000,83.09822,13.707486,37.03922,8.7580,21.9955,15.3055,NA,6.406457,1.7971273,5.048692,3.356132,NA
5,11.442533,14.018979,15.19014263,68.43942,4.794548,31.27292,7.1830,17.9830,12.0005,NA,2.671731,0.8180261,1.283082,2.180831,NA
6,19.000048,21.574621,34.79228592,53.62201,10.402874,23.70301,4.2305,14.4980,11.1905,NA,6.054450,0.6929346,3.885743,3.710080,NA
7,14.011940,16.579630,21.79739189,67.91258,7.841563,31.32608,6.6455,17.3280,12.6130,NA,3.738435,0.8736656,1.806304,2.457946,NA
8,20.603857,23.177399,40.14086533,65.64110,5.451069,28.32210,6.1305,18.6330,12.5555,NA,1.302045,0.3915248,4.860758,1.829415,NA
9,24.816652,27.380699,50.21204376,64.44568,4.767063,30.60168,5.7705,16.5155,11.5580,NA,3.426504,0.4003644,2.438529,3.052485,NA
